# <span style="color: crimson">Определение стоимости автомобилей</span>
---
**<span style="color: crimson">Заказчик</span>**: cервис по продаже автомобилей с пробегом «Не бит, не крашен».

**<span style="color: crimson">Цель анализа</span>**: построить модель по предсказанию стоимости машины по историческим данные: технические характеристики, комплектации и цены автомобилей.

**<span style="color: crimson">Датасет</span>**: технические характеристики, комплектации и цены автомобилей. 

**<span style="color: crimson">Заказчику важны:</span>**:
- качество предсказания   
- скорость предсказания   
- время обучения  
---

<h3>Описание данных:</h3>

<h4>Признаки:</h4>

* <span style="color: crimson">DateCrawled</span> — дата скачивания анкеты из базы  
* <span style="color: crimson">VehicleType</span> — тип автомобильного кузова  
* <span style="color: crimson">RegistrationYear</span> — год регистрации автомобиля  
* <span style="color: crimson">Gearbox</span> — тип коробки передач  
* <span style="color: crimson">Power</span> — мощность (л. с.)  
* <span style="color: crimson">Model</span> — модель автомобиля  
* <span style="color: crimson">Kilometer</span> — пробег (км)  
* <span style="color: crimson">RegistrationMonth</span> — месяц регистрации автомобиля  
* <span style="color: crimson">FuelType</span> — тип топлива  
* <span style="color: crimson">Brand</span> — марка автомобиля  
* <span style="color: crimson">NotRepaired</span> — была машина в ремонте или нет  
* <span style="color: crimson">DateCreated</span> — дата создания анкеты  
* <span style="color: crimson">NumberOfPictures</span> — количество фотографий автомобиля  
* <span style="color: crimson">PostalCode</span> — почтовый индекс владельца анкеты (пользователя)  
* <span style="color: crimson">LastSeen</span> — дата последней активности пользователя

<h4>Целевой признак:</h4>

* <span style="color: crimson">Price</span> — цена (евро)


---

## Импорт библиотек:

In [1]:
!pip install lightgbm
!pip install optuna
!pip install sidetable

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import sidetable as stb
import optuna
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from math import sqrt 
import pandas as pd
pd.options.mode.chained_assignment = None

In [3]:
def root_mean_squared_error(x,y):
    return sqrt(mean_squared_error(x,y))

## <span style="color: crimson">Этап 1</span> Изучение данных

<h3><span style="color: red"><b>| </b></span> Чтение данных</h3>

In [4]:
df = pd.read_csv('/datasets/autos.csv')

---

In [5]:
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Кол-во фото везде равно 0, проверим.

In [6]:
df[df['NumberOfPictures']>0].shape[0]

0

---

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

----

In [8]:
df.stb.missing(clip_0=True)\
      .style.background_gradient('coolwarm')

,missing,total,percent
NotRepaired,71154,354369,20.079070
VehicleType,37490,354369,10.579368
FuelType,32895,354369,9.282697
Gearbox,19833,354369,5.596709
Model,19705,354369,5.560588


**Пропуски:**  
Все пропуски удаляем. Несмотря на колоссальную потерю данных, во первых у нас останется около 200 тыс. объектов, а во вторых не по одну параметру мы не можем дать 100% зависимость для заполнения основываясь на иных параметрах.

---

In [9]:
df.duplicated().sum()

4

**Дубликаты:**

Кол-во дубликатов всего **4** просто удалить (хотя стоит предупредить компанию, а то  дубликаты штука подозрительная).

---

In [10]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [11]:
price_frames = df[(df["Price"] < 5)].shape[0] / df.shape[0]

print('Машин со стоимостью менее 5 евро {:.1%}:'.format(price_frames))

Машин со стоимостью менее 5 евро 3.4%:


In [12]:
year_frames = (
    df[(df["RegistrationYear"] < 1970) | (df["RegistrationYear"] > 2020)].shape[0]
    / df.shape[0]
)

print("Машин выставленных с 1970 по 2020 {:.1%}:".format(year_frames))

Машин выставленных с 1970 по 2020 0.4%:


In [13]:
power_frames = df[(df["Power"] < 1) | (df["Power"] > 1650)].shape[0] / df.shape[0]

print('Машин с кол-вом лошадиных сил более чем в болиде за 3 млн. евро {:.2%}:'.format(power_frames))

Машин с кол-вом лошадиных сил более чем в болиде за 3 млн. евро 11.40%:


**Колонки:**
1. **<span style="color: crimson">Price:</span>** минимум равен 0 (скорее всего машины бесплатно не раздают), стоит взять хотя бы от 5 евро - 4%.
2. **<span style="color: crimson">RegistrationYear:</span>** максимальный год регистрации равен 9999, а минимальный 1000, стоит поставить ограничения от 1970 до 2020. - менее 1%
3. **<span style="color: crimson">Power:</span>** мощность машины ровна 0 л.с может быть это металлолом, который как раз и отдают бесплатно, но для модели он не подходт, а максимальная немного (в 3 раза) не дотягивает до скорости ракеты. - 11%
4. **<span style="color: crimson">NumberOfPictures:</span>** кол-во фото везде равно 0

----

**Выводы:**
1. К колонок "DateCrawled", "DateCreated", "LastSeen" неверный тип данных.
2. В данных множество пропусков.
3. В данных несколько дубликатов.
4. Имеются аномалии в несколько колонках.

---

## <span style="color: crimson">Этап 2</span> Предобработка данных

#### <span style="color: crimson"> Изменение типа данных:

In [14]:
categor_columns = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']
df[categor_columns] = df[categor_columns].astype('category')

#### <span style="color: crimson"> Удалим лишние колонки:

Для модели в них не будит ни какого толка.

In [15]:
del_columns = [
    "DateCrawled",
    "DateCreated",
    "LastSeen",
    "NumberOfPictures",
    "PostalCode",
]

df = df.drop(del_columns, axis=1)

#### <span style="color: crimson"> Удалим дубликаты:

In [16]:
df = df.drop_duplicates()

#### <span style="color: crimson"> Исправим аномалии:

In [17]:
df = df[
    (df["Price"] > 200)
    & (df["RegistrationYear"] > 1970)
    & (df["RegistrationYear"] < 2020)
    & (df["Power"] < 1650)
    & (df["Power"] > 1)
]

#### <span style="color: crimson"> Удалим пропуски:

In [18]:
num_obj = df.shape[0]
df = df.dropna()
print('Удалено объектов: ',num_obj-df.shape[0])

Удалено объектов:  67121


---

## <span style="color: crimson">Этап 3</span> Обучение моделей

#### <span style="color: crimson"> Разделение выборки:

In [19]:
features = df.drop(["Price"], axis=1)
target = df["Price"]

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.40, random_state=0
)

##### Какие модели проверим? :
1. Линейная регрессия
2. Градиентный бустинг (LGBMRegressor)
3. Гребневая (ридж) регрессия
4. Регрессия по методу «лассо»
5. Регрессия «эластичная сеть»

**Настраиваем библиотеку optuna для поиска гиперпараметров**

In [20]:
RANDOM_SEED = 666

kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params

**<span style="color: crimson">|</span> Градиентный бустинг**

In [21]:
# %%time

# def LGBM_objective(trial):
#     param_grid = {
#         "num_leaves": trial.suggest_int("num_leaves", 1, 100),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 1, step=0.01),
#         "max_depth": trial.suggest_int("max_depth", 1, 10),
#         "n_estimators": trial.suggest_int("n_estimators", 10, 500),
#     }

#     LGBM = LGBMRegressor(**param_grid, random_state=RANDOM_SEED)
#     scores = cross_val_score(
#         LGBM, features_train, target_train, cv=kfolds, scoring="neg_root_mean_squared_error"
#     )
#     return scores.mean()


# LGBM_params = tune(LGBM_objective)
# LGBM = LGBMRegressor(**LGBM_params, random_state=RANDOM_SEED)

Wall time: 38min 9s

In [32]:
%%time

LGBMR_model = LGBMRegressor(
    num_leaves=99, learning_rate=0.9400000000000001, max_depth=10, n_estimators=134
).fit(features_train, target_train)
LGBMR_RMSE = root_mean_squared_error(target_test, LGBMR_model.predict(features_test))
LGBMR_RMSE

CPU times: user 12.4 s, sys: 109 ms, total: 12.5 s
Wall time: 12.6 s


1851.6991336282017

#### <span style="color: crimson"> Кодирование:

In [23]:
encoder = OrdinalEncoder()
features_train[categor_columns] = encoder.fit_transform(features_train[categor_columns])
features_test[categor_columns] = encoder.fit_transform(features_test[categor_columns])

**<span style="color: crimson">|</span> Линейная регрессия**

In [24]:
%%time
Linear_model = LinearRegression().fit(features_train, target_train)
Linear_RMSE = root_mean_squared_error(target_test, Linear_model.predict(features_test))
Linear_RMSE

CPU times: user 67.7 ms, sys: 16.8 ms, total: 84.4 ms
Wall time: 52.5 ms


2897.100166704473

**<span style="color: crimson">|</span> Гребневая (ридж) регрессия**

In [25]:
# %%time

# def ridge_objective(trial):
#     _alpha = trial.suggest_float("alpha", 0.00000001, 1)
#     ridge = Ridge(alpha=_alpha, random_state=RANDOM_SEED)
#     scores = cross_val_score(
#         ridge, features_train, target_train, cv=kfolds,
#         scoring="neg_root_mean_squared_error"
#     )
#     return scores.mean()

# ridge_params = tune(ridge_objective)
# ridge = Ridge(**ridge_params, random_state=RANDOM_SEED)

Wall time: 24.4 s

In [26]:
%%time

Ridge_model = Ridge(alpha=0.9998822938507691).fit(features_train, target_train)
Ridge_RMSE = root_mean_squared_error(target_test, Ridge_model.predict(features_test))
Ridge_RMSE

CPU times: user 6.95 ms, sys: 32.2 ms, total: 39.2 ms
Wall time: 18.4 ms


2897.0996061010947

**<span style="color: crimson">|</span> Регрессия по методу «лассо»**

In [27]:
# %%time

# def lasso_objective(trial):
#     _alpha = trial.suggest_float("alpha", 0.0000001, 1)
#     lasso = Lasso(alpha=_alpha, random_state=RANDOM_SEED)
#     scores = cross_val_score(
#         lasso, features_train, target_train, cv=kfolds,
#         scoring="neg_root_mean_squared_error"
#     )
#     return scores.mean()

# lasso_params = tune(lasso_objective)
# lasso = Lasso(**lasso_params, random_state=RANDOM_SEED)

Wall time: 48.1 s

In [28]:
%%time

lasso_model = Lasso(random_state=0, alpha=0.15631666743004888).fit(features_train, target_train)
lasso_RMSE = root_mean_squared_error(target_test, lasso_model.predict(features_test))
lasso_RMSE

CPU times: user 247 ms, sys: 120 ms, total: 368 ms
Wall time: 394 ms


2897.0879762726413

**<span style="color: crimson">|</span> Регрессия «эластичная сеть»** 

In [29]:
# %%time

# def ElasticNet_objective(trial):
#     _alpha = trial.suggest_float("alpha", 0.00000001, 1)
#     ElNet = ElasticNet(alpha=_alpha, random_state=RANDOM_SEED)
#     scores = cross_val_score(
#         ElNet,
#         features_train,
#         target_train,
#         cv=kfolds,
#         scoring="neg_root_mean_squared_error",
#     )
#     return scores.mean()


# ElasticNet_params = tune(ElasticNet_objective)
# ElNET = ElasticNet(**ElasticNet_params, random_state=RANDOM_SEED)

Wall time: 42.7 s

In [30]:
%%time

ElNet_model = ElasticNet(random_state=0, alpha=0.00014232563970705304).fit(features_train, target_train)
ElNet_RMSE = root_mean_squared_error(target_test, ElNet_model.predict(features_test))
ElNet_RMSE

CPU times: user 258 ms, sys: 129 ms, total: 387 ms
Wall time: 385 ms


2897.095161039118

## <span style="color: crimson">Этап 4</span>  Общий вывод

### Анализ моделей:

In [31]:
pd.DataFrame(
    [
        ["Линейная регрессия", Linear_RMSE, "Wall time: 36.8 ms", "0"],
        ["Градиентный бустинг", LGBMR_RMSE, "Wall time: 38min 9s", "Wall time: 3.17 s"],
        [
            "Гребневая (ридж) регрессия",
            Ridge_RMSE,
            "Wall time: 24.4 s",
            "Wall time: 23.9 ms",
        ],
        [
            "Регрессия по методу «лассо»",
            lasso_RMSE,
            "Wall time: 48.1 s",
            "Wall time: 45.9 ms",
        ],
        [
            "Регрессия «эластичная сеть»",
            ElNet_RMSE,
            "Wall time: 42.7 s",
            "Wall time: 54.9 ms",
        ],
    ],
    columns=("Модель", " RMSE", "Время предсказание", "Время поиска гиперпараметров"),
)

,Модель,RMSE,Время предсказание,Время поиска гиперпараметров
0,Линейная регрессия,2897.100167,Wall time: 36.8 ms,0
1,Градиентный бустинг,1118.660337,Wall time: 38min 9s,Wall time: 3.17 s
2,Гребневая (ридж) регрессия,2897.099606,Wall time: 24.4 s,Wall time: 23.9 ms
3,Регрессия по методу «лассо»,2897.087976,Wall time: 48.1 s,Wall time: 45.9 ms
4,Регрессия «эластичная сеть»,2897.095161,Wall time: 42.7 s,Wall time: 54.9 ms


Как очевидно из представленных результатов, лучшее качество у модели градиентного бустинга, хотя время поиска гиперпараметров и предсказания (по сравнению с остальными моделями) очень высоки.

### Советы:
1. К каждой машине добавить vin код для того, чтобы можно было восстановить недостающие данные.
2. Не допускать пропусков в параметрах, так как такие данные приходится удалять (особенно если отсутствует целевой признак).
3. В признаке кол-во фото у всех машин стоит 0, возможна ошибка сбора данных.